In [ ]:
# DenseNet121을 이용한 애니메이션 표정 분류 Fine-tuning 실험

# 1. 필요한 라이브러리 임포트 및 환경 설정
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime

# Google Drive 마운트 및 기본 설정
drive.mount('/content/drive')

# 필요한 패키지 설치
%pip install torch torchvision tqdm pandas matplotlib

# 프로젝트 디렉토리 설정
project_path = '/content/drive/MyDrive/Final'
os.chdir(project_path)

# 디렉토리 구조 설정
dirs = {
    'model': os.path.join(project_path, 'model'),
    'result': os.path.join(project_path, 'result'),
    'plots': os.path.join(project_path, 'plots'),
    'data': {
        'train': os.path.join(project_path, 'ani/train'),
        'val': os.path.join(project_path, 'ani/val')
    }
}

# 필요한 디렉토리 생성
for dir_path in [dirs['model'], dirs['result'], dirs['plots']]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# 장치 설정
device = torch.device('cuda')

# 2. 데이터 전처리 및 증강 설정
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# 데이터셋 로드
train_dataset = datasets.ImageFolder(dirs['data']['train'], transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(dirs['data']['val'], transform=data_transforms['val'])

# 클래스 정보 출력
print("\n데이터셋 정보:")
print("클래스:", train_dataset.classes)
print("클래스 수:", len(train_dataset.classes))
print("학습 데이터 수:", len(train_dataset))
print("검증 데이터 수:", len(val_dataset))

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# 3. DenseNet121 모델 설정 및 학습
def setup_densenet121(num_classes):
    """DenseNet121 모델 설정"""
    model = models.densenet121(pretrained=True)
    
    # 특성 추출기의 파라미터 고정
    for param in model.parameters():
        param.requires_grad = False
    
    # 분류기 층 수정
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    )
    
    return model.to(device)

def train_model(model, criterion, optimizer, num_epochs=25):
    """모델 학습 함수"""
    best_acc = 0.0
    results = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # 학습 단계
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(train_loader, desc='Training'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)
        results['train_loss'].append(epoch_loss)
        results['train_acc'].append(epoch_acc.item())
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # 검증 단계
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(val_loader, desc='Validation'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects.double() / len(val_dataset)
        results['val_loss'].append(epoch_loss)
        results['val_acc'].append(epoch_acc.item())
        
        print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # 최고 성능 모델 저장
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
                'acc': epoch_acc,
            }, os.path.join(dirs['model'], f'densenet121_best_{timestamp}.pt'))
        

    
    # 최종 결과 저장
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pd.DataFrame(results).to_csv(os.path.join(dirs['result'], f'densenet121_results_{timestamp}.csv'))
    
    return results

# 모델 학습 실행
print("\nDenseNet121 모델 학습 시작")
model = setup_densenet121(len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

results = train_model(model, criterion, optimizer)

# 4. 결과 시각화
def plot_training_results(results):
    """학습 결과 시각화"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 전체 그래프 (Loss와 Accuracy)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 손실 그래프
    ax1.plot(results['train_loss'], label='Train')
    ax1.plot(results['val_loss'], label='Validation')
    ax1.set_title('Loss over epochs')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    
    # 정확도 그래프
    ax2.plot(results['train_acc'], label='Train')
    ax2.plot(results['val_acc'], label='Validation')
    ax2.set_title('Accuracy over epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    
    plt.tight_layout()
    
    # 전체 그래프 저장
    plt.savefig(os.path.join(dirs['plots'], f'densenet121_combined_{timestamp}.png'))
    plt.show()
    
    # Loss 그래프 따로 저장
    plt.figure(figsize=(8, 5))
    plt.plot(results['train_loss'], label='Train')
    plt.plot(results['val_loss'], label='Validation')
    plt.title('Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['plots'], f'densenet121_loss_{timestamp}.png'))
    plt.close()
    
    # Accuracy 그래프 따로 저장
    plt.figure(figsize=(8, 5))
    plt.plot(results['train_acc'], label='Train')
    plt.plot(results['val_acc'], label='Validation')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['plots'], f'densenet121_accuracy_{timestamp}.png'))
    plt.close()

# 결과 시각화 및 최종 성능 출력
print("\n학습 결과:")
plot_training_results(results)
print(f"\n최종 성능:")
print(f"최종 학습 정확도: {results['train_acc'][-1]:.4f}")
print(f"최종 검증 정확도: {results['val_acc'][-1]:.4f}")
print(f"최고 검증 정확도: {max(results['val_acc']):.4f}")
